# PyTorch_Learningn (MIVRC)

![](https://ws1.sinaimg.cn/large/abaebc48ly1fqgrsuu355j209x02wjr7.jpg)

## 课程6：卷积神经网络

### 代码示例：构建一个3层的卷积神经网络对MNIST进行分类
包含2个卷积层和一个全连接层

In [1]:
#导入所需的库/包
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
#定义超参
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [3]:
#下载 MNIST 数据集
train_dataset = datasets.MNIST(root='./MNIST/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root = './MNIST/', train=False, transform=transforms.ToTensor(), download=True)

In [4]:
#加载数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

![](https://ws1.sinaimg.cn/large/abaebc48ly1fqi1ktqfe0j20ne06fjs1.jpg)

In [5]:
#构建CNN网络模型
#写法一：

#torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        # 输出out是包含batchsize维度为4的tensor，即(batchsize，channels，x，y)
        # out.size(0)指batchsize的值
        # out = out.view(x.size(0), -1)简化 x = out.view(batchsize, -1)
        # view()函数的功能根reshape类似，用来转换size大小。out = out.view(batchsize, -1)中batchsize指转换后有几行，而-1指在不告诉函数有多少列的情况下，根据原tensor数据和batchsize自动分配列数。
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [6]:
# 加载构建好的网络模型并调用GPU加速
cnn = CNN()
cnn.cuda()

CNN (
  (layer1): Sequential (
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU ()
    (3): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (fc): Linear (1568 -> 10)
)

In [7]:
#构建CNN网络模型
#写法二：

class CNN_new(nn.Module):
    def __init__(self):
        super(CNN_new, self).__init__()
        self.relu = nn.ReLU()
        self.Pool = nn.MaxPool2d(2)
        self.layer1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, padding=2)
        self.layer2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=2)
        self.fc = nn.Linear(7*7*32, 10)
        self.BatchNorm1 = nn.BatchNorm2d(16)
        self.BatchNorm2 = nn.BatchNorm2d(32)   
        
    def forward(self, x):
        out = self.Pool(self.relu(self.BatchNorm1(self.layer1(x))))
        out = self.Pool(self.relu(self.BatchNorm2(self.layer2(out))))
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [8]:
# 加载构建好的网络模型并调用GPU加速
cnn_new = CNN_new()
cnn_new.cuda()

CNN_new (
  (relu): ReLU ()
  (Pool): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (layer1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (layer2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc): Linear (1568 -> 10)
  (BatchNorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (BatchNorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
)

In [9]:
#定义损失函数
criterion = nn.CrossEntropyLoss()
#定义优化器
optimizer = torch.optim.Adam(cnn_new.parameters(), lr=learning_rate)

In [10]:
#开始训练网络
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        
        #调用模型进行前向传播
        outputs = cnn_new(images)
        
        #调用损失函数计算输出结果与真实值的差
        loss = criterion(outputs, labels)
        
        #启用优化器并将梯度缓存清空，再进行反向传播
        optimizer.zero_grad()
        loss.backward()
        
        #调用优化函数开始迭代优化
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/10], Iter [100/600] Loss: 0.1965
Epoch [1/10], Iter [200/600] Loss: 0.1001
Epoch [1/10], Iter [300/600] Loss: 0.0920
Epoch [1/10], Iter [400/600] Loss: 0.1218
Epoch [1/10], Iter [500/600] Loss: 0.1498
Epoch [1/10], Iter [600/600] Loss: 0.0261
Epoch [2/10], Iter [100/600] Loss: 0.0855
Epoch [2/10], Iter [200/600] Loss: 0.0914
Epoch [2/10], Iter [300/600] Loss: 0.0186
Epoch [2/10], Iter [400/600] Loss: 0.0033
Epoch [2/10], Iter [500/600] Loss: 0.0096
Epoch [2/10], Iter [600/600] Loss: 0.0173
Epoch [3/10], Iter [100/600] Loss: 0.0493
Epoch [3/10], Iter [200/600] Loss: 0.0540
Epoch [3/10], Iter [300/600] Loss: 0.0336
Epoch [3/10], Iter [400/600] Loss: 0.0253
Epoch [3/10], Iter [500/600] Loss: 0.0648
Epoch [3/10], Iter [600/600] Loss: 0.0262
Epoch [4/10], Iter [100/600] Loss: 0.0082
Epoch [4/10], Iter [200/600] Loss: 0.0513
Epoch [4/10], Iter [300/600] Loss: 0.0227
Epoch [4/10], Iter [400/600] Loss: 0.0034
Epoch [4/10], Iter [500/600] Loss: 0.0389
Epoch [4/10], Iter [600/600] Loss:

In [13]:
#测试网络测试
cnn_new.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images).cuda()
    outputs = cnn_new(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99 %


In [14]:
#保存网络模型及参数
torch.save(cnn.state_dict(), 'cnn.pkl')